In [1]:
#installing ollama
#!pip install ollama
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [2]:
#starting ollama server locally
import subprocess
import time
process = subprocess.Popen("ollama serve", shell=True)
time.sleep(5)  # Wait for 5 seconds

In [3]:
#pulling llama3 using ollama
!ollama pull llama3

pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏  13 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   1% ▕▏  36 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   1% ▕▏  68 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   2% ▕▏ 100 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   3% ▕▏ 117 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   3% ▕▏ 151 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   4% ▕▏ 194 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   5% ▕▏ 212 MB/4.7 GB             

In [4]:
#testing llama 3 is availble
!ollama list

NAME             ID              SIZE      MODIFIED               
llama3:latest    365c0bd3c000    4.7 GB    Less than a second ago    


In [5]:
!pip install langchain_community langchain langchain-openai langchain_pinecone langchain[docarray] docarray pydantic==1.10.8 pytube python-dotenv tiktoken pinecone-client scikit-learn ruff pypdf faiss-cpu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.4/146.4 kB 6.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to pr

In [6]:
import os
from langchain_community.vectorstores import FAISS
from langchain_community.llms import Ollama
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.embeddings import OllamaEmbeddings
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import DocArrayInMemorySearch
from operator import itemgetter

In [7]:

#setting up model dynamically
#from dotenv import load_dotenv

#load_dotenv()

#OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
#MODEL = "gpt-3.5-turbo"
#MODEL = "mixtral:8x7b"
MODEL = "llama3"

#VECTOR_STORE = "default"
VECTOR_STORE = "FAISS"
#VECTOR_STORE = "pinecone"

In [60]:
#loading model based on selection
if MODEL.startswith("gpt"):
    model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=MODEL)
    embeddings = OpenAIEmbeddings()
else:
    model = Ollama(model=MODEL, temperature = 0.2)
    embeddings = OllamaEmbeddings(model=MODEL)

#testing model invoke
model.invoke("what is ML")

"ML stands for Machine Learning, which is a subfield of artificial intelligence (AI) that involves training algorithms to make predictions or decisions based on data.\n\nMachine learning is a type of self-improving system that can learn from experience and improve its performance over time. It's different from traditional programming, where the rules are hardcoded by humans. In machine learning, the algorithm learns from the data it's trained on, without being explicitly programmed for each specific task.\n\nThe goal of machine learning is to develop algorithms that can:\n\n1. **Learn**: From a dataset or experience.\n2. **Reason**: Make predictions or decisions based on what they've learned.\n3. **Improve**: Over time, by refining their performance through feedback and iteration.\n\nMachine learning has many applications in various fields, including:\n\n1. **Computer Vision**: Image recognition, object detection, facial recognition, etc.\n2. **Natural Language Processing (NLP)**: Text

In [61]:
#creating parser
parser = StrOutputParser()

chain = model | parser

#testing model invoke with parser
chain.invoke("what is ML")

"ML can refer to several things, but I'll assume you're asking about Machine Learning (ML).\n\nMachine learning is a subfield of artificial intelligence (AI) that involves training algorithms to make predictions or decisions based on data. The goal is to enable machines to learn from experience and improve their performance over time without being explicitly programmed.\n\nIn machine learning, an algorithm is trained on a dataset, which consists of input data (features) and corresponding output labels (target variable). The algorithm learns patterns and relationships within the data by adjusting its parameters or weights. This process is called training.\n\nOnce trained, the algorithm can be used to make predictions or decisions on new, unseen data. Machine learning models can be applied to various tasks, such as:\n\n1. **Classification**: Predicting a class label (e.g., spam/not spam email) based on input features.\n2. **Regression**: Predicting a continuous value (e.g., stock price) 

In [62]:
#creating template
template = """
Analyze the provided financial data, focusing on the specified financial metric and its impact.

When conducting your analysis, consider the following:

1.  **Key Figures:** State the key financial figures for the current and prior periods relevant to the specified metric.
2.  **Change Analysis:** Calculate and state the year-over-year (y/y) change in the metric, including both absolute values and percentages.
3.  **Explanation of Changes:** Provide concise and specific reasons for the observed changes in the financial metric.

In addition, incorporate the following into your analysis as relevant:

-   **Related Ratios or Metrics:** Include any relevant financial ratios or metrics that provide further context or insights into the changes in the primary metric. Analyze their trends (stable, increased, decreased).
-   **Impact on Profitability:** If applicable, discuss the impact of the changes in the financial metric on profitability (e.g., net operating profit, gross profit).

Ensure the following:
-   Use specific numerical values, including dollar amounts and percentages, where applicable.
-   Provide concise and specific reasons for changes.
-   Avoid unnecessary details.
-   Present your analysis in a clear and concise manner.

Financial Data: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)

#testing prompt
prompt.format(context="Here is some context", question="Here is a question")

'\nAnalyze the provided financial data, focusing on the specified financial metric and its impact.\n\nWhen conducting your analysis, consider the following:\n\n1.  **Key Figures:** State the key financial figures for the current and prior periods relevant to the specified metric.\n2.  **Change Analysis:** Calculate and state the year-over-year (y/y) change in the metric, including both absolute values and percentages.\n3.  **Explanation of Changes:** Provide concise and specific reasons for the observed changes in the financial metric.\n\nIn addition, incorporate the following into your analysis as relevant:\n\n-   **Related Ratios or Metrics:** Include any relevant financial ratios or metrics that provide further context or insights into the changes in the primary metric. Analyze their trends (stable, increased, decreased).\n-   **Impact on Profitability:** If applicable, discuss the impact of the changes in the financial metric on profitability (e.g., net operating profit, gross prof

In [63]:
#loading pdf files
import copy

loader = PyPDFLoader("/content/gxocompany.pdf")
pages = loader.load_and_split()
pages_original = copy.deepcopy(pages)
pages_original[35]

Document(metadata={'source': '/content/gxocompany.pdf', 'page': 24}, page_content='Basis of Presentation\nOn August 2, 2021, the Company became a standalone publicly traded company and its financial statements post the separation from XPO, Inc. (the “Separation”) are now prepared on a consolidatedbasis. Before the Separation, GXO’s historical financial statements were prepared on a standalone combined basis and were derived from the Consolidated Financial Statements and accountingrecords of XPO. The combined Consolidated Financial Statements for all periods presented before the Separation are now referred to as “Consolidated Financial Statements” and have been preparedunder the U.S. generally accepted accounting principles (“GAAP”). GXO has one reportable segment.\nOn October 23, 2023, the Company completed the acquisition of PFSweb, Inc. (“PFS”), an e-commerce order fulfillment company based in Irving, Texas (the “PFS Acquisition”). On May 24, 2022,the Company completed the acquisitio

In [64]:
# Define the page numbers you want to extract
target_page_numbers = [35,36]  # Example page numbers

# Create an empty string to store the combined content
combined_content = ""

# Iterate through the pages and extract content from the target pages
page_number = 0 # Start with page number 1
for page in pages_original:
    if page_number in target_page_numbers:
        combined_content += page.page_content
        # You can add a separator (e.g., "\n\n") between page contents if needed
    page_number += 1 # Increment the page number for the next page

# Print or use the combined content
print(combined_content)

Basis of Presentation
On August 2, 2021, the Company became a standalone publicly traded company and its financial statements post the separation from XPO, Inc. (the “Separation”) are now prepared on a consolidatedbasis. Before the Separation, GXO’s historical financial statements were prepared on a standalone combined basis and were derived from the Consolidated Financial Statements and accountingrecords of XPO. The combined Consolidated Financial Statements for all periods presented before the Separation are now referred to as “Consolidated Financial Statements” and have been preparedunder the U.S. generally accepted accounting principles (“GAAP”). GXO has one reportable segment.
On October 23, 2023, the Company completed the acquisition of PFSweb, Inc. (“PFS”), an e-commerce order fulfillment company based in Irving, Texas (the “PFS Acquisition”). On May 24, 2022,the Company completed the acquisition of Clipper Logistics plc (“Clipper”), an omnichannel retail logistics specialist ba

In [65]:
import re
# Clean unnecessary data in pages
for page in pages:
    # Remove headers and footers (example patterns)
    page.page_content = re.sub(r"Header text pattern", "", page.page_content)
    page.page_content = re.sub(r"Footer text pattern", "", page.page_content)

    # Remove specific patterns (example)
    page.page_content = re.sub(r"© \d{4} Company Name", "", page.page_content)

    # Remove page numbers (example)
    page.page_content = re.sub(r"\d+/\d+", "", page.page_content)

    # Remove special characters (except for alphanumeric, spaces, $, ., and ,)
    page.page_content = re.sub(r"[^a-zA-Z0-9 \$,.]", "", page.page_content) # Added , and . to the excluded characters

In [66]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Adjust based on document size
    chunk_overlap=200,  # Ensures important context isn't lost
    separators=["\n\n", "\n", " "]  # Prioritize splitting at paragraph breaks
)

#chunks = text_splitter.split_documents(pages)
chunks = text_splitter.create_documents([combined_content])

In [67]:
import pandas as pd
from langchain.schema import Document

df = pd.read_excel('/content/gxo_statement.xlsx')

all_content = ""
separator = "\n\n"  # Define your separator (e.g., newline characters)
for index, row in df.iterrows():
    row_content = ' '.join([str(cell) for cell in row.values])  # Combine all cell values in a row
    all_content += row_content + separator  # Add row content and separator to the combined string


document = Document(page_content=all_content)
chunks_excel = text_splitter.split_documents([document])

print(chunks_excel)

all_chunks = chunks_excel + chunks

[Document(page_content='statement date 12/312023 2022-12-31 00:00:00\n\nstatement length 12 Months 12 Months\n\nRevenue 9778 8993\n\nTotal Cost of Goods Sold (COGS) 7443 8035\n\nGross Profit 1743 1550\n\nTransaction and integration costs 34 61\n\nTotal operating expenses 1532 1343\n\nNet operating profit 353 275')]


In [68]:
#loading vector store based on selection
if VECTOR_STORE == "FAISS":
    #vectorstore = FAISS.from_documents(pages, embeddings)
    vectorstore = FAISS.from_documents(all_chunks, embeddings)
else:
    vectorstore = DocArrayInMemorySearch.from_documents(chunks, embedding=embeddings, verbose=True)

In [69]:
#setting up retriver
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

#testing retriver
retriever.invoke("balance sheets")

[Document(page_content='Transaction and integration costs were $34 million in 2023, compared with $61 million for 2022. Transaction and integration costs in 2023 included $20 million related to the integration of Clipperand $12 million related to the PFS Acquisition, reflecting costs associated with advisory fees and severance costs. Transaction and integration costs in 2022 primarily related to $46 million related tothe Clipper Acquisition, reflecting costs associated with financing arrangements, advisory fees and integration costs, and $15 million from the Separation, primarily reflecting rebranding costs.'),
 Document(page_content='On October 23, 2023, the Company completed the acquisition of PFSweb, Inc. (“PFS”), an e-commerce order fulfillment company based in Irving, Texas (the “PFS Acquisition”). On May 24, 2022,the Company completed the acquisition of Clipper Logistics plc (“Clipper”), an omnichannel retail logistics specialist based in Leeds, England (the “Clipper Acquisition”

In [70]:
#all_text = ""
#for page in pages:
#    all_text += page.page_content

In [71]:
def print_context(inputs):
  print("Context:")
  print(inputs["context"])
  return inputs

In [72]:
#chaining all operations
from langchain_core.runnables import RunnableLambda

chain = (
    {
        "context": itemgetter("question") | retriever, #| RunnableLambda(lambda x: all_text),
        #"context": RunnableLambda(lambda x: combined_content),
        "question": itemgetter("question"),
    }
    | RunnableLambda(print_context)
    | prompt
    | model
    | parser
)

In [79]:
#posting questions
questions = [
    #"provide finanacial analysis as underwriter in financial company by going to complete context that has in company annual reports",
     #"how is the revenue growth of company in 2023",
   """
   Analyze the financial statements and provide a structured summary of Revenue growth for the current and prior periods.
Your response should include:
1. Total revenue for the current and prior periods.
2. Year-over-year (y/y) change in absolute ($) and percentage (%) terms.
3. Key factors contributing to the revenue change
   """,
"""
Analyze the 'transactions and integration costs' from the provided financial data.

Your analysis should include the following:

1. **Key Figures:** State the total 'transactions and integration costs' for the current and prior periods.
2. **Change Analysis:** Calculate and state the year-over-year (y/y) change in 'transactions and integration costs,' including both the absolute value and percentage change.
3. **Explanation of Changes:** Provide a detailed explanation for the observed change in 'transactions and integration costs,' including specific reasons and any contributing factors.
    * If possible, break down the change by identifying specific components of 'transactions and integration costs' and their individual contributions to the overall change. For example, mention changes in integration costs related to specific acquisitions or projects.

Present your analysis in a clear and concise manner, using specific numerical values, dollar amounts, and percentages wherever applicable.
""",
"""
Analyze the 'operating expenses' from the provided financial data, focusing on the operating expense ratio, net operating profit, and revenue.

Your analysis should include the following:

1. Key Figures: State the total 'operating expenses' for the current and prior periods. Include a breakdown of specific contributing factors and their impact on net operating profit.
    * **What is the operating expense ratio for the current and prior periods? How does the trend of operating expenses compare to revenue growth?**
2. Change Analysis: Calculate and state the year-over-year (y/y) change in 'operating expenses,' including both the absolute value and percentage change.
    * **What is the year-over-year change in net operating profit? refer net operating profit from income statement **
3. Explanation of Changes: Provide a detailed explanation for the observed change in 'operating expenses,' including specific reasons and any contributing factors.

Present your analysis in a clear and concise manner, using specific numerical values, dollar amounts, and percentages wherever applicable.
""",
"""
Analyze the financial statements provided and summarize the key financial performance for the recent fiscal year, focusing on revenue, cost of goods sold (COGS), gross profit, and gross profit margin.

Your analysis should include:

* **Key Figures:** Present the key financial figures for the current and prior periods for each of the metrics mentioned, including specific amounts and percentages. Show your calculations to derive these figures where possible, using the data provided in the context.
* **Change Analysis:** Calculate and state the year-over-year (y/y) change for each metric, including both absolute values ($) and percentages (%).
* **Explanation of Changes:** Provide concise and specific reasons for the observed changes in each metric, referencing the context and highlighting key factors driving the performance.
* **Related Ratios and Metrics:** Include relevant financial ratios or metrics that provide further context (e.g., gross profit margin). Analyze their trends (stable, increased, decreased) and their impact on profitability.
* **Impact on Profitability:** Discuss the overall impact of the changes on the company's profitability, highlighting key drivers and trends.

Present your analysis in a clear and concise manner, using bullet points or numbered lists for clarity. Support your analysis with specific numerical values, dollar amounts, and percentages wherever applicable.
"""

]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

Question: 
   Analyze the financial statements and provide a structured summary of Revenue growth for the current and prior periods.
Your response should include:
1. Total revenue for the current and prior periods.
2. Year-over-year (y/y) change in absolute ($) and percentage (%) terms.
3. Key factors contributing to the revenue change
   
Context:
[Document(page_content='On October 23, 2023, the Company completed the acquisition of PFSweb, Inc. (“PFS”), an e-commerce order fulfillment company based in Irving, Texas (the “PFS Acquisition”). On May 24, 2022,the Company completed the acquisition of Clipper Logistics plc (“Clipper”), an omnichannel retail logistics specialist based in Leeds, England (the “Clipper Acquisition”). Due to the acquisitions ofClipper and PFS, comparisons in our results of operations between 2023 and 2022 are less meaningful.'), Document(page_content='Transaction and integration costs were $34 million in 2023, compared with $61 million for 2022. Transaction and 